In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import codecs
from __future__ import print_function

# Communication to TensorFlow server via gRPC
from grpc.beta import implementations
import tensorflow as tf
import numpy as np

# TensorFlow serving stuff to send messages
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
from scipy.io import loadmat
from copy import deepcopy
from flask import send_file
from itertools import groupby
from tqdm import tqdm
import random 
import pandas as pd

/home/olivier/anaconda3/envs/1.5_cpu/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
SERVER_NAME, SERVER_PORT = "0.0.0.0", "9000"

VOCAB_BASED_LANGUAGES = ["en"]

LANGUAGE = "fr"
MODEL_NAME = "w2p_" + LANGUAGE

# if LANGUAGE in VOCAB_BASED_LANGUAGES:
#     VOCAB = "../api/word_to_phonetic/"+LANGUAGE+"/vocab"
# else:
VOCAB = None
    
DATASET = "../data_dir/"+LANGUAGE+"_2/test_dataset.csv" 

In [14]:
def _char_encode(input, padding_to = None):
    """
    Transform txt input to int tokens
    +2 is for special tokens ["<EOS>", "<PAD>"]
    + [1] is to add end of sequence "<EOS>" token

    :param input: String input
    :return: [1, -1, 1, 1] Int array
    """
    inp = [c + 2 for c in input.encode("Latin-1")] + [1]
    if padding_to:
        for _ in range(padding_to - len(inp)):
            inp += [0]
    inp = np.reshape(inp, [1, -1, 1, 1])

    return inp

def _char_decode(input):
    """
    Decode token ids to string and removes padding and eos

    :param input: int array
    :return: String
    """

    return [chr(idx - 2) for idx in input if idx > 1]

def _vocab_encode(input, vocab, padding_to = None):
    with open(vocab, "r") as f:
        vocab_arr = [l.strip()[1:-1] for l in f.readlines()]
    
    try :
        inp = [np.where(np.array(vocab_arr) == (c.upper() + "_"))[0][0] for c in input] + [1]
    except :
        print(input)
    if padding_to:
        for _ in range(padding_to - len(inp)):
            inp += [0]
    inp = np.reshape(inp, [1, -1, 1, 1])
    return inp

def _vocab_decode(input, vocab):
    with open(vocab, "r") as f:
        vocab_arr = [l.strip()[1:-1] for l in f.readlines()]
    return [vocab_arr[i][:-1] for i in input if i>1]

def _create_translate_request(input, model_name):
    """
    Creates translate request to TensorFlow serving server

    :param input: Int array, token ids
    :param model_name: Name of the model to serve
    :return: PredictRequest object
    """
    # create predict request
    request = predict_pb2.PredictRequest()

    # Call model to make phonetic translation
    request.model_spec.name = model_name
    request.model_spec.signature_name = "get_phon"
    request.inputs['input'].CopyFrom(
        tf.contrib.util.make_tensor_proto(input, dtype=tf.int32))

    return request


def _open_tf_server_channel(server_name, server_port):
    """
    Opens channel to TensorFlow server for requests

    :param server_name: String, server name (localhost, IP address)
    :param server_port: String, server port
    :return: Channel stub
    """
    channel = implementations.insecure_channel(
        server_name,
        int(server_port))
    stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

    return stub

def _format_phon(phon):
    values = phon.int_val
    shape1 = phon.tensor_shape.dim[0].size
    shape2 = phon.tensor_shape.dim[1].size

    return np.reshape(values, [shape1, shape2])


def _make_translation(input, model_name, stub):
    translation_request = _create_translate_request(input, model_name)
    translation_results = stub.Predict(translation_request, 60.0)
    return _format_phon(translation_results.outputs["phon"])


def _make_translation_batch(batch_input, model_name, stub, vocab=None):
    padding_to = len(max(batch_input, key=len)) + 1
    
    if not vocab:
        batch_input_tokenized = np.stack([_char_encode(input, padding_to).squeeze(0) for input in batch_input], 0)

    else:
        batch_input_tokenized = np.stack([_vocab_encode(input, vocab, padding_to).squeeze(0) for input in batch_input], 0)

    batch_phon_tokenized = _make_translation(batch_input_tokenized, model_name, stub)

    if not vocab:
        batch_phon = ["".join(_char_decode(phon_tokenized)) for phon_tokenized in batch_phon_tokenized]
    else:
        batch_phon = ["".join(_vocab_decode(phon_tokenized, vocab)) for phon_tokenized in batch_phon_tokenized]
    
    return batch_phon




In [4]:
txt,phon = [], []
with open(DATASET, "r") as f:
    for l in f.readlines():
        l = l.strip().split(",")
        txt.append(l[0])
        phon.append(l[1].replace(" ", ""))

In [7]:
_char_encode("test")

array([[[[118]],

        [[103]],

        [[117]],

        [[118]],

        [[  1]]]])

In [15]:
# open channel to tensorflow server
stub = _open_tf_server_channel(SERVER_NAME, SERVER_PORT)

# get phonetic translation and attention matrices
batch_size = 64
wordCount = len(txt)
n_batch = wordCount // batch_size

pred_phon = []

for idx_batch in tqdm(range(n_batch + 1)):
    try:
        batch = txt[idx_batch * batch_size:(idx_batch + 1) * batch_size]
    except:
        batch = txt[n_batch * batch_size:]
    
    batch_phon = _make_translation_batch(batch, MODEL_NAME, stub, VOCAB)
    pred_phon.extend(batch_phon)

100%|██████████| 79/79 [00:39<00:00,  2.02it/s]


In [20]:
import random
rand = random.randint(0,len(txt))
print("{} // {} : {}".format(txt[rand], pred_phon[rand], phon[rand]))

twopenny // twopeni : t-xpxn-i


In [7]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 
            deletions = current_row[j] + 1       
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def per(s1,s2):
    return levenshtein(s1,s2) / len(s1)

def wer(s1,s2):
    return int(s1!=s2)

In [8]:
def error_rates(s,p):
    perS = 0
    werS = 0
    for i, _ in tqdm(enumerate(s)):
        werS += wer(s[i],p[i])
        perS += per(s[i],p[i])
    return werS/len(s), perS/len(s)

In [9]:
rates = error_rates(phon,pred_phon)
print("WER : {:.4%} ; PER : {:.4%}".format(rates[0], rates[1]))

4002it [00:00, 32762.24it/s]

WER : 29.5852% ; PER : 6.5942%
